In [1]:
import redis
import pandas as pd
from datetime import datetime

db = redis.Redis(host='127.0.0.1', port=6379, db=0, charset="utf-8", decode_responses=True)
#db.flushdb()

In [5]:
gastosG= pd.read_csv('data/GastosExtra.csv')
ingresosi= pd.read_csv('data/IngresosExtra.csv')
ingresosi

,Fecha,Concepto,Ingreso,Usuario
0,2022-06-05 20:08:36,Prepago,50,HPPS125982733
1,2022-06-05 20:09:54,Devoluciones,33,HPPS125982733
2,2022-06-05 20:10:01,Prepago,7,HPPS125982733
3,2022-06-05 20:10:01,Prepago,31,HPPS125982733
4,2022-06-05 20:10:11,Prepago,73,HSDP02837162
...,...,...,...,...
98,2022-06-05 20:14:48,Devoluciones,61,MMCL10294873
99,2022-06-05 20:14:48,Anticipo,76,HSDP02837162
100,2022-06-05 20:14:49,Emergencias,20,MMCL10294873
101,2022-06-05 20:14:50,Devoluciones,41,HPPS125982733


In [2]:
def carga_datosPrueba(pruebasG, pruebasI):
    key1G='gastoExtra:Gasto:'
    key2G='gastoExtra:Concepto:'
    key3G='gastoExtra:Hora:'
    key4G='gastoExtra:Usuario:'

    key1I='ingresoExtra:Ingreso:'
    key2I='ingresoExtra:Concepto:'
    key3I='ingresoExtra:Hora:'
    key4I='ingresoExtra:Usuario:'
    for index, row in pruebasG.iterrows():
        db.rpush(key1G, row[2])
        db.rpush(key2G, row[1])
        db.rpush(key3G, row[0])
        db.rpush(key4G, row[3])
        
    for index, row in pruebasI.iterrows():
        db.rpush(key1I, row[2])
        db.rpush(key2I, row[1])
        db.rpush(key3I, row[0])
        db.rpush(key4I, row[3])
    

def carga_productos(restantes):
    for index, row in restantes.iterrows():
        key= 'Productos:'+row[0]
        result= db.set(key, int(row[1]))
    return result

def get_sobrantes(producto):
    key="Productos:"+producto
    result = db.get(key)
    return int(result)

def producidos(producto, producido):
    key="Productos:"+producto
    result= db.set(key, get_sobrantes(producto)+producido)
    return result

def comprados(producto, comprado):
    key="Productos:"+producto
    result= db.set(key, get_sobrantes(producto)-comprado)
    return result

def gastosExtra(gastado, concepto, user):
    hora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    key1='gastoExtra:Gasto:'
    key2='gastoExtra:Concepto:'
    key3='gastoExtra:Hora:'
    key4='gastoExtra:Usuario:'
    db.rpush(key1, gastado)
    db.rpush(key2, concepto)
    db.rpush(key3, hora)
    db.rpush(key4, user)
    return 

def gastosExtraTotales():
    gastos= db.lrange('gastoExtra:Gasto:', 0, -1)
    return sum(map(int, gastos))

def ingresosExtra(ingresado, concepto, user):
    hora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    key1='ingresoExtra:Ingreso:'
    key2='ingresoExtra:Concepto:'
    key3='ingresoExtra:Hora:'
    key4='ingresoExtra:Usuario:'
    db.rpush(key1, ingresado)
    db.rpush(key2, concepto)
    db.rpush(key3, hora)
    db.rpush(key4, user)
    return

def dfIngresosExtra(self):
        tiempos=[]
        for i in self.db.lrange('ingresoExtra:Hora:', 0, -1):
            tiempos.append(datetime.strptime(i, "%Y-%m-%d %H:%M:%S"))
        ingdict={'Fecha': tiempos, 'Concepto':self.db.lrange('ingresoExtra:Concepto:', 0, -1), \
            'Ingreso': self.db.lrange('ingresoExtra:Ingreso:', 0, -1), 'Usuario': self.db.lrange('ingresoExtra:Usuario:', 0, -1)}
        ingdf=pd.DataFrame.from_dict(data=ingdict)
        return( ingdf)

def dfGastosExtra():
        tiempos=[]
        for i in db.lrange('gastoExtra:Hora:', 0, -1):
            tiempos.append(datetime.strptime(i, "%Y-%m-%d %H:%M:%S"))
        gastdict= {'Fecha': tiempos, 'Concepto':db.lrange('gastoExtra:Concepto:', 0, -1), \
            'Costo': db.lrange('gastoExtra:Gasto:', 0, -1), 'Usuario': db.lrange('gastoExtra:Usuario:', 0, -1)}
        gastodf=pd.DataFrame.from_dict(data=gastdict)
        return(gastodf)




In [4]:
get_sobrantes("Volovan Jamon")

10

In [7]:
carga_datosPrueba(gastosG, ingresosi)

In [8]:
k=dfGastosExtra()
k

,Fecha,Concepto,Costo,Usuario
0,2022-06-05 20:08:37,Detergente,81,MTYSJ12476412
1,2022-06-05 20:09:55,Manzanas,22,MTYSJ12476412
2,2022-06-05 20:09:57,Manzanas,59,HSDP02837162
3,2022-06-05 20:10:00,Manzanas,71,MMCL10294873
4,2022-06-05 20:10:03,Charloa,18,HPPS125982733
...,...,...,...,...
94,2022-06-05 20:14:45,Aceite,95,HPPS125982733
95,2022-06-05 20:14:49,Manzanas,28,MMCL10294873
96,2022-06-05 20:14:50,Manzanas,68,MTYSJ12476412
97,2022-06-05 20:14:50,Quita Grasa,31,MMCL10294873
